In [0]:
df = spark.read.option("header", True).csv("/FileStore/tables/ipl_commentary_data.csv")


In [0]:

df = df.filter(col("year").cast("int").isNotNull())
df = df.filter((col("year") >= 2008) & (col("year") <= 2025))


In [0]:

df = df.filter(col("team1_name").isNotNull() & col("team2_name").isNotNull())


In [0]:

df = df.dropDuplicates(["match_id", "team1_name", "team2_name"])


In [0]:
df.select("year", "team1_name", "team2_name") \
  .withColumnRenamed("team1_name", "team") \
  .select("year", "team") \
  .union(df.select("year", col("team2_name").alias("team")).select("year", "team")) \
  .distinct() \
  .groupBy("year") \
  .agg(countDistinct("team").alias("number_of_teams")) \
  .orderBy("year") \
  .show()


+----+---------------+
|year|number_of_teams|
+----+---------------+
|2017|              8|
|2018|              8|
|2019|              8|
|2020|              8|
|2021|              9|
|2022|             10|
|2023|             10|
|2024|             10|
|2025|             10|
+----+---------------+



In [0]:
df.select("match_id", "match_status") \
  .dropDuplicates(["match_id"]) \
  .groupBy("match_status") \
  .agg(count("*").alias("num_matches")) \
  .show()


+------------+-----------+
|match_status|num_matches|
+------------+-----------+
|   completed|        516|
|        tied|          8|
+------------+-----------+



In [0]:
df.select("match_id", "over_no", "ball_no") \
  .dropDuplicates(["match_id", "over_no", "ball_no"]) \
  .groupBy("match_id", "over_no") \
  .agg(count("ball_no").alias("balls_in_over")) \
  .groupBy("match_id") \
  .agg(avg("balls_in_over").alias("avg_balls_per_inning")) \
  .agg(avg("avg_balls_per_inning").alias("overall_avg_balls_per_inning")) \
  .show()


+----------------------------+
|overall_avg_balls_per_inning|
+----------------------------+
|          0.9980916030534351|
+----------------------------+



In [0]:
df.select("year", "match_id", "match_winning_team") \
  .dropDuplicates(["match_id"]) \
  .groupBy("year", "match_winning_team") \
  .agg(count("*").alias("wins")) \
  .orderBy("match_winning_team", "year") \
  .show()


+----+-------------------+----+
|year| match_winning_team|wins|
+----+-------------------+----+
|2018|Chennai Super Kings|  11|
|2019|Chennai Super Kings|  10|
|2020|Chennai Super Kings|   6|
|2021|Chennai Super Kings|  11|
|2022|Chennai Super Kings|   4|
|2023|Chennai Super Kings|  10|
|2024|Chennai Super Kings|   7|
|2025|Chennai Super Kings|   1|
|2019|     Delhi Capitals|   9|
|2020|     Delhi Capitals|   8|
|2021|     Delhi Capitals|   9|
|2022|     Delhi Capitals|   7|
|2023|     Delhi Capitals|   5|
|2024|     Delhi Capitals|   7|
|2025|     Delhi Capitals|   1|
|2017|   Delhi Daredevils|   6|
|2018|   Delhi Daredevils|   5|
|2017|      Gujarat Lions|   4|
|2022|     Gujarat Titans|  12|
|2023|     Gujarat Titans|  11|
+----+-------------------+----+
only showing top 20 rows



In [0]:
df_batsman = df.withColumn("batsman", regexp_extract(col("ball_commentary"), r"to ([^,]+),", 1))

df_batsman.groupBy("match_id", "team1_name") \
          .agg(collect_list("batsman").alias("batsmen_list"),
               collect_set("batsman").alias("batsmen_set")) \
          .show(truncate=False)


+--------+----------+-----------------+-----------------+
|match_id|team1_name|batsmen_list     |batsmen_set      |
+--------+----------+-----------------+-----------------+
|114960  |KKR       |[de Kock]        |[de Kock]        |
|114967  |SRH       |[Abhishek Sharma]|[Abhishek Sharma]|
|114976  |MI        |[Rohit]          |[Rohit]          |
|114985  |LSG       |[Markram]        |[Markram]        |
|114987  |PBKS      |[Priyansh Arya]  |[Priyansh Arya]  |
|114996  |RR        |[Jaiswal]        |[Jaiswal]        |
|115005  |SRH       |[Head]           |[Head]           |
|115012  |RCB       |[Phil Salt]      |[Phil Salt]      |
|18121   |SRH       |[Warner]         |[Warner]         |
|18122   |MI        |[Parthiv Patel]  |[Parthiv Patel]  |
|18123   |GL        |[Roy]            |[Roy]            |
|18124   |RPS       |[Rahane]         |[Rahane]         |
|18125   |RCB       |[Gayle]          |[Gayle]          |
|18126   |GL        |[Roy]            |[Roy]            |
|18127   |KKR 